In [4]:
# from catboost import CatBoostClassifier

import pandas as pd
import os
import warnings
import numpy as np
import re

from functools import reduce

warnings.filterwarnings("ignore")

In [11]:
data_columns = ['bodyType', 'brand', 'car_url', 'color', 'engineDisplacement', 'enginePower', 
                      'equipment_dict','fuelType', 'mileage', 'modelDate', 'model_name', 'numberOfDoors', 
                      'productionDate', 'sell_id', 'vehicleTransmission', 'vendor', 
                      'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль']

# =============================================================

In [5]:
# обьединение данных по каждом бренду в один DataFrame
# by_brend_dir = os.listdir('C:/Users/Manych/projects/persnl/Project_7_data_extra/by_brend')

# to_concat = []
# for brand in by_brend_dir:
#     try: 
#         brand_df = pd.read_csv(
#             'C:/Users/Manych/projects/persnl/Project_7_data_extra/by_brend/' + brand,sep=',',low_memory=False)
#     except:
#         brand_df = pd.read_csv(
#             'C:/Users/Manych/projects/persnl/Project_7_data_extra/by_brend/' + brand,sep=';',low_memory=False)
#     to_concat.append(brand_df)

# consolidated_df = pd.concat(to_concat)

# consolidated_df.to_csv('C:/Users/Manych/projects/persnl/Project_7_data_extra/consolidated_df.csv')

In [6]:
# all_dfs_names = [x for x in os.listdir('C:/Users/Manych/projects/persnl/Project_7_data_extra') if '.csv' in x]
all_dfs_names = [x for x in os.listdir('./Preproject_7_data') if '.csv' in x]

# Получение серий со спискоми колонок каждого датафрейма, рколичества наблюдений в каждом датафрейме и с самими датафремами

In [42]:
dfs_columns = pd.Series()
# dfs_counts = pd.Series()
# all_dfs =  pd.Series()
for df_name in  all_dfs_names:
    print(df_name,end=' ')
    try:
        df = pd.read_csv('./Preproject_7_data/' + df_name,low_memory=False)
    except:
        df = pd.read_csv('./Preproject_7_data/' + df_name,sep='\t',low_memory=False) 
    dfs_columns = dfs_columns.append(pd.Series([list(df.columns)],index=[df_name]))
#     dfs_counts = dfs_counts.append(pd.Series([len(df)],index=[df_name]))
#     all_dfs = all_dfs.append(pd.Series([df],index=[df_name]));
#     print(df.columnsumns)

all_auto_ru_09_09_2020.csv all_auto_ru_14_11_2020.csv all_car_27_08_2021_data_12_brands.csv cars_df_big.csv cars_df_small.csv data1.csv old_train.csv parsing_auto_ru.csv train_df_full_part1.csv 

In [41]:
# [x for x in dfs_columns.index[[0,2,5,7,8]]]
# [set(data_columns) <= set(x) for x in dfs_columns]
# {1,2,3} <= {1,2,3,4}
# [set(data_columns) - set(x) for x in dfs_columns]
# [ set(x)- set(data_columns) for x in dfs_columns]
# [ 'parsing_unixtime' in set(x) for x in dfs_columns]

In [39]:
datasets_to_train = ['all_auto_ru_09_09_2020.csv',
 'all_car_27_08_2021_data_12_brands.csv',
 'data1.csv',
 'parsing_auto_ru.csv',
 'train_df_full_part1.csv']

# Общие для всех  датафреймов колонки, и остальные(необщий)

In [8]:
# '''
common_columns = reduce(lambda x,y: set(x)&set(y), dfs_columns)
not_common_columns = dfs_columns.apply(lambda x: set(x) - common_columns )
[print (sorted(x)) for x in not_common_columns];
# common_columns
# not_common_columns
# '''

['hidden', 'model', 'start_date', 'Владение', 'Комплектация', 'Состояние', 'Таможня']
['complectation_dict', 'equipment_dict', 'image', 'model_info', 'model_name', 'priceCurrency', 'price_EUR', 'price_USD', 'sell_id', 'start_date', 'super_gen', 'vendor', 'Владение', 'Состояние', 'Таможня']
['Unnamed: 0', 'car_url', 'complectation_dict', 'date_added', 'equipment_dict', 'image', 'model_name', 'parsing_unixtime', 'priceCurrency', 'region', 'sell_id', 'super_gen', 'views', 'Владение', 'Состояние', 'Таможня']
['car_url', 'equipmentGroups', 'image', 'model_name', 'parsing_unixtime', 'priceCurrency', 'sell_id', 'Состояние', 'Таможня']
['car_url', 'equipmentGroups', 'image', 'model_name', 'parsing_unixtime', 'priceCurrency', 'sell_id', 'Состояние', 'Таможня']
['car_url', 'complectation_dict', 'equipment_dict', 'image', 'is_train', 'model_info', 'model_name', 'parsing_unixtime', 'priceCurrency', 'sell_id', 'super_gen', 'vendor', 'Владение', 'Состояние', 'Таможня']
['complectation_dict', 'equipm

In [9]:
total_dfs_columns = pd.Series(dfs_columns.sum()).value_counts()

In [10]:
total_dfs_columns[:30]

Руль                    9
color                   9
enginePower             9
price                   9
fuelType                9
mileage                 9
vehicleTransmission     9
Владельцы               9
modelDate               9
name                    9
ПТС                     9
vehicleConfiguration    9
brand                   9
description             9
numberOfDoors           9
bodyType                9
Привод                  9
engineDisplacement      9
productionDate          9
priceCurrency           8
Состояние               8
Таможня                 8
model_name              8
sell_id                 8
image                   7
parsing_unixtime        6
car_url                 6
super_gen               6
complectation_dict      6
Владение                6
dtype: int64

# Взвешенные по количеству наблюдений колонки 

In [8]:
weighted_columns = dfs_counts*dfs_columns
total_weighted_columns = pd.Series(weighted_columns.sum()).value_counts()

In [9]:
total_weighted_columns[:20]

Владельцы              1139446
description            1139446
name                   1139446
fuelType               1139446
ПТС                    1139446
price                  1139446
brand                  1139446
enginePower            1139446
color                  1139446
bodyType               1139446
mileage                1139446
vehicleTransmission    1139446
productionDate         1139446
Руль                   1139446
Привод                 1139446
engineDisplacement     1139446
model_name             1050068
sell_id                1050068
equipment_dict          981588
Таможня                 972048
dtype: int64

In [10]:
# [print (sorted(x)) for x in not_common_columns]

# ===================all_dfs_getting===============================

In [11]:
all_dfs =  pd.Series()

for df_name in  all_dfs_names:
    print(df_name,end=' ')
    try:
        df = pd.read_csv('./Preproject_7_data/' + df_name,low_memory=False)
    except:
        df = pd.read_csv('./Preproject_7_data/' + df_name,sep='\t',low_memory=False)
        
    all_dfs = all_dfs.append(pd.Series([df],index=[df_name]))

all_auto_ru_09_09_2020.csv all_auto_ru_14_11_2020.csv all_car_27_08_2021_data_12_brands.csv cars_df_big.csv cars_df_small.csv consolidated_df.csv data1.csv parsing_auto_ru.csv train.csv train_df_full_part1.csv 

In [12]:
for df_name,df in zip(all_dfs.index,all_dfs):
    try:
        print(df[df.brand=='MERCEDES'].price.mean(),
              df_name)
    except: 
        print(df[df.brand=='MERCEDES'].price.apply(lambda x: int(''.join(re.findall("\d+", x))) if type(x)==str else np.NaN).mean(),
              df_name)
              

3526259.7003383883 all_auto_ru_09_09_2020.csv
4879901.107416625 all_auto_ru_14_11_2020.csv
6093548.989583893 all_car_27_08_2021_data_12_brands.csv
3435923.9598136283 cars_df_big.csv
3435923.9598136283 cars_df_small.csv
nan consolidated_df.csv
2385619.3733265977 data1.csv
2306780.622999633 parsing_auto_ru.csv
4509805.734230299 train.csv
5869825.628195463 train_df_full_part1.csv


# =============== dfs columns study ===========================

# =============== brends study ===============================

In [13]:
for df_name,df in zip(all_dfs[1:].index,all_dfs[1:]):
        print(df.brand.unique(),
              df_name)

['AUDI' 'BMW' 'CADILLAC' 'CHERY' 'CHEVROLET' 'CHRYSLER' 'CITROEN' 'DAEWOO'
 'DODGE' 'FORD' 'GEELY' 'HONDA' 'HYUNDAI' 'INFINITI' 'JAGUAR' 'JEEP' 'KIA'
 'LEXUS' 'MAZDA' 'MINI' 'MITSUBISHI' 'NISSAN' 'OPEL' 'PEUGEOT' 'PORSCHE'
 'RENAULT' 'SKODA' 'SUBARU' 'SUZUKI' 'TOYOTA' 'VOLKSWAGEN' 'VOLVO'
 'GREAT_WALL' 'LAND_ROVER' 'MERCEDES' 'SSANG_YONG'] all_auto_ru_14_11_2020.csv
['SKODA' 'AUDI' 'VOLVO' 'BMW' 'NISSAN' 'INFINITI' 'MERCEDES' 'TOYOTA'
 'LEXUS' 'VOLKSWAGEN' 'MITSUBISHI' 'HONDA'] all_car_27_08_2021_data_12_brands.csv
['SKODA' 'AUDI' 'HONDA' 'VOLVO' 'BMW' 'NISSAN' 'INFINITI' 'MERCEDES'
 'TOYOTA' 'LEXUS' 'VOLKSWAGEN' 'MITSUBISHI'] cars_df_big.csv
['SKODA' 'AUDI' 'HONDA' 'VOLVO' 'BMW' 'NISSAN' 'INFINITI' 'MERCEDES'
 'TOYOTA' 'LEXUS' 'VOLKSWAGEN' 'MITSUBISHI'] cars_df_small.csv
['Audi' 'BMW' 'Chery' nan 'Citroen' 'Ford' 'ГАЗ' 'Geely' 'Haval' 'Honda'
 'Hyundai' 'Infiniti' 'Kia' 'Land Rover' 'Lexus' 'Lifan' 'Mazda'
 'Mercedes-Benz' 'Mitsubishi' 'Nissan' 'Opel' 'Peugeot' 'Porsche'
 'Renault' 'S

# =============== models study ===============================

In [14]:
brand = 'BMW'

In [15]:
brand_models=pd.Series()   

for df_name,df in zip(all_dfs[1:].index,all_dfs[1:]):
    brand_models = brand_models.append(pd.Series([df[df.brand==brand].model_name.value_counts()],
                                                 index=[df_name]))

### распределения цены по моделям

## 0

In [16]:
models_0 = {'3ER': '3 СЕРИИ','5ER': '5 СЕРИИ','X5': 'X5','X6': 'X6','X3': 'X3','7ER': '7 СЕРИИ',
 'X1': 'X1','X7': 'X7','1ER': '1 СЕРИИ','X4': 'X4','X6_M': 'X6 M','6ER': '6 СЕРИИ',
 '2ER': '2 СЕРИИ','X5_M': 'X5 M','4': '4 СЕРИИ','8ER': '8 СЕРИИ','M5': 'M5','X2': 'X2',
 'I3': 'I3 ','Z4': 'Z4','M3': 'M3','M6': 'M6','M2': 'M2','M4': 'M4',
 '2GRANDTOURER': '2 СЕРИИ GRAN TOURER','I8': 'I8','M8': 'M8','321': '321',
 '2ACTIVETOURER': '2 СЕРИИ ACTIVE TOURER','Z3M': 'Z3 M','X3_M': 'X3 M','326': '326',
 '02': '02','340': '340','E3': 'E3','Z1': 'Z1','Z3': 'Z3'}
# brand_models.iloc[0]

In [17]:
# brand_models.iloc[0]
# [print("'" + x + "': ''") for x in brand_models.iloc[0].index];
# [print(x) for x in brand_models.iloc[1].index];

In [18]:
# brand_models.iloc[0]
unificated_model_name_0 = all_dfs[1][all_dfs[1].brand == 'BMW'].model_name.map(models_0)
unificated_model_name_0

3642         02
3643    1 СЕРИИ
3644    1 СЕРИИ
3645    1 СЕРИИ
3646    1 СЕРИИ
         ...   
9752         Z4
9753         Z4
9754         Z4
9755         Z4
9756         Z4
Name: model_name, Length: 6115, dtype: object

In [19]:
unificated_model_name_0.value_counts()[:10]

3 СЕРИИ    1229
5 СЕРИИ     943
X5          849
X6          571
X3          569
7 СЕРИИ     385
X1          271
X7          215
1 СЕРИИ     199
X4          165
Name: model_name, dtype: int64

## 1

In [20]:
# brand_models.iloc[1]
all_dfs[2][all_dfs[2].brand == 'BMW'].model_name.str.upper()

5013           X5
5014          NaN
5015           X3
5016          NaN
5017      5 СЕРИИ
           ...   
116767         X6
116768        NaN
116769         X4
116770         X1
116771         X1
Name: model_name, Length: 16497, dtype: object

In [21]:
all_dfs[2][all_dfs[2].brand == 'BMW'].model_name.str.upper().value_counts()[:10]

X5         1968
5 СЕРИИ    1457
1 СЕРИИ    1433
X1         1294
X4         1228
X6         1143
3 СЕРИИ    1051
X3          779
X5 M        432
X6 M        412
Name: model_name, dtype: int64

## 2

In [22]:
# brand_models.iloc[2]
# [print(x) for x in brand_models.iloc[2].index[-55:]];
# (?<=BMW )(.*(?= \d)|\d*|.*(?= \w))

In [23]:
unificated_model_name_2 = all_dfs[3][all_dfs[3].brand == 'BMW'].model_name.apply(
    lambda x: re.findall("(?<=BMW )(.*(?= \d)|\d*|.*(?= \w))", x)[-1]).str.upper()
unificated_model_name_2

11068    4 СЕРИИ GRAN COUPE
11069               3 СЕРИИ
11070               3 СЕРИИ
11071               5 СЕРИИ
11072               3 СЕРИИ
                ...        
14533               5 СЕРИИ
14534                  X5 M
14535               5 СЕРИИ
14536               3 СЕРИИ
14537                    X5
Name: model_name, Length: 3470, dtype: object

In [24]:
unificated_model_name_2.value_counts()[:10]

5 СЕРИИ         690
3 СЕРИИ         494
X5              480
X6              291
X3              281
X1              148
7 СЕРИИ LONG    126
7 СЕРИИ         104
X4               98
1 СЕРИИ          79
Name: model_name, dtype: int64

In [25]:
# models_2 = {'X5': 'X5','X6': 'X6','5': '5 серии','3': '3 серии','X3': 'X3','X7': 'X7','1': '1 серии',
#  'X4': 'X4','X1': 'X1','7': '7 серии','6': '6 серии','8': '8 серии','M5': 'M5','4': '4 серии',
#  'X2': 'X2','2': '2 серии','i3': 'I3','M3': 'M3','M4': 'M4','Z4': 'Z4','M2': 'M2','321': '321',
#  'M6': 'M6','340': '340','502': '502','Z1': 'Z1','M8': 'M8','Z3': 'Z3','600': '600','2000': '2000'}

## 3

In [26]:
# brand_models.iloc[3]

In [27]:
unificated_model_name_3 = all_dfs[4][all_dfs[4].brand == 'BMW'].model_name.apply(
    lambda x: re.findall("(?<=BMW )(.*(?= \d)|\d*|.*(?= \w))", x)[-1]).str.upper()
unificated_model_name_3

11068    4 СЕРИИ GRAN COUPE
11069               3 СЕРИИ
11070               3 СЕРИИ
11071               5 СЕРИИ
11072               3 СЕРИИ
                ...        
14533               5 СЕРИИ
14534                  X5 M
14535               5 СЕРИИ
14536               3 СЕРИИ
14537                    X5
Name: model_name, Length: 3470, dtype: object

In [28]:
unificated_model_name_3.value_counts()[:10]

5 СЕРИИ         690
3 СЕРИИ         494
X5              480
X6              291
X3              281
X1              148
7 СЕРИИ LONG    126
7 СЕРИИ         104
X4               98
1 СЕРИИ          79
Name: model_name, dtype: int64

## 4 

In [29]:
all_dfs[5][all_dfs[5].brand == 'BMW'].model_name.str.upper()
# brand_models.iloc[4]

11268                3 СЕРИИ
11269                     I8
11270    2 СЕРИИ GRAN TOURER
11271                     Z3
11272                     X3
                ...         
25633                7 СЕРИИ
25634                     X5
25635                     X3
25636                     X7
25637                3 СЕРИИ
Name: model_name, Length: 14370, dtype: object

In [30]:
all_dfs[5][all_dfs[5].brand == 'BMW'].model_name.str.upper().value_counts()[:10]

5 СЕРИИ    2548
3 СЕРИИ    2490
X5         2200
7 СЕРИИ    1204
X3         1189
X6         1099
X1          694
1 СЕРИИ     496
X4          413
6 СЕРИИ     312
Name: model_name, dtype: int64

## 5 

In [31]:
models_5 = {'7 СЕРИИ': '7 СЕРИИ','02 (E10)': '02','1 СЕРИИ': '1 СЕРИИ','2 СЕРИИ ACTIVE TOURER': '2 СЕРИИ ACTIVE TOURER',
'5 СЕРИИ': '5 СЕРИИ','M1': 'M1','X3 M': 'X3 M','Z3 M': 'Z3 M','X4_M': 'X4 M','4 СЕРИИ': '4 СЕРИИ',
 '2 СЕРИИ': '2 СЕРИИ','X5 M': 'X5 M','8 СЕРИИ': '8 СЕРИИ','6 СЕРИИ': '6 СЕРИИ','3 СЕРИИ': '3 СЕРИИ',
 'X6 M': 'X6 M','2 СЕРИИ GRAND TOURER': '2 СЕРИИ GRAND TOURER'}

models_0.update(models_5)

In [32]:
all_dfs[6][all_dfs[6].brand == 'BMW'].model_name.map(models_0)

7477          X5
7478          X6
7479          X6
7480          X7
7481          X6
          ...   
68055    5 СЕРИИ
68056         X6
68057         X4
68058         X5
69000    5 СЕРИИ
Name: model_name, Length: 8203, dtype: object

In [33]:
all_dfs[6][all_dfs[6].brand == 'BMW'].model_name.map(models_0).value_counts()[:10]

5 СЕРИИ    1657
3 СЕРИИ    1472
X5         1213
7 СЕРИИ     598
X6          592
X3          568
X1          396
1 СЕРИИ     353
X4          228
6 СЕРИИ     215
Name: model_name, dtype: int64

## 6 

In [34]:
all_dfs[7][all_dfs[7].brand == 'BMW'].model_name.map(models_0)

1459      5 СЕРИИ
1460      5 СЕРИИ
1461      5 СЕРИИ
1462           X3
1463           X5
           ...   
113311    5 СЕРИИ
113320    3 СЕРИИ
113321    3 СЕРИИ
113324    7 СЕРИИ
113325    3 СЕРИИ
Name: model_name, Length: 12847, dtype: object

In [35]:
all_dfs[7][all_dfs[7].brand == 'BMW'].model_name.map(models_0).value_counts()[:10]

5 СЕРИИ    3051
3 СЕРИИ    2248
X5         1926
X6         1058
X3          995
7 СЕРИИ     903
X1          535
1 СЕРИИ     467
6 СЕРИИ     297
X4          290
Name: model_name, dtype: int64

## 7

In [36]:
all_dfs[8][all_dfs[8].brand == 'BMW'].model_name.map(models_0)

37        3 СЕРИИ
38             X3
39        5 СЕРИИ
40             X6
41             X3
           ...   
167326    3 СЕРИИ
167327         X5
167328         X3
167329    5 СЕРИИ
167330    5 СЕРИИ
Name: model_name, Length: 5959, dtype: object

In [37]:
all_dfs[8][all_dfs[8].brand == 'BMW'].model_name.map(models_0)

37        3 СЕРИИ
38             X3
39        5 СЕРИИ
40             X6
41             X3
           ...   
167326    3 СЕРИИ
167327         X5
167328         X3
167329    5 СЕРИИ
167330    5 СЕРИИ
Name: model_name, Length: 5959, dtype: object

## 8

In [38]:
all_dfs[9][all_dfs[9].brand == 'BMW'].model_name.str.upper().value_counts()[:10]

5 СЕРИИ    2337
X3         1343
3 СЕРИИ    1339
X1         1074
X5          861
7 СЕРИИ     795
X6          731
6 СЕРИИ     674
X4          418
X7          358
Name: model_name, dtype: int64

# =============== test study ==================================

In [39]:
test = pd.read_csv('./Project_7_data/test.csv')

In [40]:
test.columns

Index(['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
       'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency',
       'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'Состояние', 'Таможня'],
      dtype='object')

In [41]:
nun_count = test.isna().sum()
nun_count[nun_count>0]

complectation_dict    28268
equipment_dict         9996
Владение              22691
ПТС                       1
dtype: int64

In [42]:
test.brand.unique()

array(['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
       'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI'],
      dtype=object)

# ====================== ПОДВАЛ =============================

Если печать идёт как в ДОСе (текст отсылается в устройство PRN), то перевод строки обеспечивается управляющим символом с ASCII-кодом 12 ♀.
При печати через PCL/PS - напечатается изображение символа. Так что, всё зависит от того, как работает с текстом проприетарное ПО...
Если идёт печать через PCL/PS, то "спрятать" в тексте команду перевода страницы не получится, эту команду должно подавать само ПО.